# Q1) Quantum Teleportation & No-Cloning

## 1. Theoretical Derivation of the Teleportation Protocol

Quantum teleportation allows the transfer of a quantum state $|\psi\rangle = \alpha|0\rangle + \beta|1\rangle$ from one location (Alice) to another (Bob) using a shared entangled pair and classical communication.

### Step 1: Initialization
The system consists of three qubits:
- Qubit $q_0$: Alice's qubit to be teleported, in state $|\psi\rangle = \alpha|0\rangle + \beta|1\rangle$.
- Qubit $q_1$: Alice's half of the Bell pair.
- Qubit $q_2$: Bob's half of the Bell pair.

The shared entangled Bell pair is in the state:
$$ |\Phi^+\rangle_{12} = \frac{1}{\sqrt{2}} (|00\rangle + |11\rangle) $$

The total state of the system is $|\Psi\rangle_0 = |\psi\rangle_0 \otimes |\Phi^+\rangle_{12}$:
$$ |\Psi\rangle_0 = \frac{1}{\sqrt{2}} (\alpha|0\rangle + \beta|1\rangle)_0 \otimes (|00\rangle + |11\rangle)_{12} $$
$$ |\Psi\rangle_0 = \frac{1}{\sqrt{2}} (\alpha|000\rangle + \alpha|011\rangle + \beta|100\rangle + \beta|111\rangle) $$

### Step 2: CNOT Gate
Alice applies a CNOT gate with $q_0$ as control and $q_1$ as target.
$$ \text{CNOT}_{0\to1} |\Psi\rangle_0 = \frac{1}{\sqrt{2}} (\alpha|000\rangle + \alpha|011\rangle + \beta|110\rangle + \beta|101\rangle) $$

### Step 3: Hadamard Gate
Alice applies a Hadamard gate to $q_0$.
Recall $H|0\rangle = \frac{|0\rangle+|1\rangle}{\sqrt{2}}$ and $H|1\rangle = \frac{|0\rangle-|1\rangle}{\sqrt{2}}$.
$$ |\Psi\rangle_2 = \frac{1}{2} [ \alpha(|0\rangle+|1\rangle)(|00\rangle+|11\rangle) + \beta(|0\rangle-|1\rangle)(|10\rangle+|01\rangle) ] $$

Regrouping terms by Alice's qubits ($q_0, q_1$):
$$ |\Psi\rangle_2 = \frac{1}{2} [ |00\rangle(\alpha|0\rangle + \beta|1\rangle) + |01\rangle(\alpha|1\rangle + \beta|0\rangle) + |10\rangle(\alpha|0\rangle - \beta|1\rangle) + |11\rangle(\alpha|1\rangle - \beta|0\rangle) ] $$

### Step 4: Measurement and Correction
Alice measures qubits $q_0$ and $q_1$, obtaining two classical bits $M_0$ and $M_1$. The state of Bob's qubit $q_2$ collapses based on the measurement:

| Alice's Measure ($M_1 M_0$) | Bob's State ($q_2$) | Action Required |
| :--- | :--- | :--- |
| 00 | $\alpha|0\rangle + \beta|1\rangle$ | Identity ($I$) |
| 01 | $\alpha|1\rangle + \beta|0\rangle$ | Pauli-$X$ ($X$) |
| 10 | $\alpha|0\rangle - \beta|1\rangle$ | Pauli-$Z$ ($Z$) |
| 11 | $\alpha|1\rangle - \beta|0\rangle$ | $ZX$ |

**Final State of Bob's Qubit $q_2$:**
After applying the appropriate corrections ($X^{M_1} Z^{M_0}$), Bob's qubit is restored to the original state:
$$ |\psi\rangle_{Bob} = \alpha|0\rangle + \beta|1\rangle $$

## 2. Analysis: The Fate of Alice's State and No-Cloning

**The No-Cloning Theorem** states that it is impossible to create an independent and identical copy of an arbitrary unknown quantum state.
$$ U(|\psi\rangle \otimes |0\rangle) \neq |\psi\rangle \otimes |\psi\rangle $$

In the teleportation protocol, this theorem is upheld. When Alice performs the Bell basis measurement on her qubits ($q_0$ and $q_1$), the quantum information is effectively "disassembled." 
*   Alice's qubit $q_0$ does **not** retain the state $|\psi\rangle$. It collapses into a classical basis state ($|0\rangle$ or $|1\rangle$).
*   The original state $|\psi\rangle$ is destroyed at Alice's end before it is reconstructed at Bob's end.
*   Therefore, the state is **transferred**, not cloned. The information is transmitted via the classical channel (the measurement results) and the non-local correlation of entanglement, respecting the no-cloning restriction.


In [ ]:
# Implementation of Quantum Teleportation using Qiskit
from qiskit import QuantumCircuit, Aer, transpile, assemble, execute
from qiskit.visualization import plot_histogram, plot_bloch_multivector
import numpy as np

# 1. Initialize the Quantum Circuit
# 3 Qubits and 3 Classical Bits
qc = QuantumCircuit(3, 3)

# 2. Prepare Alice's Qubit (q0) with a random state or specific rotation
# Let's rotate q0 to a specific state to verify later
# Example: Ry(pi/3) rotation
qc.ry(np.pi/3, 0)
qc.barrier()

# 3. Create Bell Pair (Entanglement) between Alice (q1) and Bob (q2)
qc.h(1)
qc.cx(1, 2)
qc.barrier()

# 4. Bell Basis Measurement (Alice's Operations)
qc.cx(0, 1)
qc.h(0)
qc.barrier()

# Measure Alice's qubits
qc.measure(0, 0)
qc.measure(1, 1)
qc.barrier()

# 5. Conditional Corrections (Bob's Operations)
# Apply X if q1 measured 1
qc.x(2).c_if(1, 1)
# Apply Z if q0 measured 1
qc.z(2).c_if(0, 1)

# 6. Verify Correct Teleportation
# We measure Bob's qubit to see if it matches Alice's initial statistical distribution
qc.measure(2, 2)

# Draw the circuit
qc.draw(output='mpl')


In [ ]:
# Initial State Verification (Theoretical)
# For Ry(pi/3): 
# alpha = cos(pi/6) = sqrt(3)/2 approx 0.866
# beta = sin(pi/6) = 0.5
# Prob(0) = 0.75, Prob(1) = 0.25

# Execute Simulation
backend = Aer.get_backend('qasm_simulator')
job = execute(qc, backend, shots=1024)
result = job.result()
counts = result.get_counts(qc)

# Filter counts to look at Bob's measurement (Bit 2)
# The keys are 'cum2 cumb1 cumb0'. We care about the most significant bit (Bob's)
bobs_counts = {'0': 0, '1': 0}
for state, count in counts.items():
    if state[0] == '0':
        bobs_counts['0'] += count
    else:
        bobs_counts['1'] += count

print("Teleportation Results (Bob's Qubit):")
print(f"Measured counts: {bobs_counts}")
print(f"Expected Probability ratio ~ 3:1 for |0> vs |1> check")
plot_histogram(bobs_counts)


# Q2) Diffie-Hellman Man-in-the-Middle (MitM) Attack
    
## 1. Vulnerability Analysis

The **Diffie-Hellman (DH)** key exchange protocol is secure against eavesdropping (passive attacks) due to the difficulty of the Discrete Logarithm Problem (DLP). However, it is vulnerable to an active **Man-in-the-Middle (MitM)** attack because the standard protocol **lacks authentication**.

**The Attack Mechanism:**
1.  **Interception**: An attacker, Mallory, sits between Alice and Bob.
2.  **Impersonation**:
    *   When Alice sends her public key $A$ to Bob, Mallory intercepts it and sends her own public key $M$ to Bob.
    *   When Bob sends his public key $B$ to Alice, Mallory intercepts it and sends her own public key $M$ to Alice.
3.  **Key Establishment**:
    *   Alice computes a secret shared with Mallory (thinking it is Bob).
    *   Bob computes a secret shared with Mallory (thinking it is Alice).
    *   Mallory can now decrypt, read, modify, and re-encrypt messages between Alice and Bob without them knowing.


In [ ]:
# Simulation of DH MitM Attack

def power(a, b, P):
    if (b == 1):
        return a
    else:
        return ((pow(a, b)) % P)

# --- Standard Parameters ---
P = 23  # Prime number
G = 5   # Primitive root

print(f"Publicly Shared Variables:\n P = {P}\n G = {G}\n")

# --- Alice's Perspective ---
a = 6  # Alice's private key
x = power(G, a, P) # Alice's generated key (A)
print(f"Alice: Private Key a = {a}")
print(f"Alice: Generates Public Key A = {x}")
print("Alice sends A to 'Bob' (Intercepted by Mallory)...\n")

# --- Mallory's Interception (Alice -> Bob) ---
# Mallory blocks A and sends her own public key M
m = 10 # Mallory's private key
M = power(G, m, P) # Mallory's public key
print(f"[MALLORY]: Intercepts A. Private Key m = {m}")
print(f"[MALLORY]: Sends Fake Public Key M = {M} to Bob\n")

# --- Bob's Perspective ---
# Bob receives M (thinking it is Alice's key A)
b = 15 # Bob's private key
y = power(G, b, P) # Bob's generated key (B)
print(f"Bob: Private Key b = {b}")
print(f"Bob: Generates Public Key B = {y}")
print("Bob sends B to 'Alice' (Intercepted by Mallory)...\n")

# --- Mallory's Interception (Bob -> Alice) ---
# Mallory blocks B and sends her own public key M to Alice
print(f"[MALLORY]: Intercepts B.")
print(f"[MALLORY]: Sends Fake Public Key M = {M} to Alice\n")

# --- Secret Key Generation ---

# Alice computes secret (Using M, thinking it is Bob's B)
ka = power(M, a, P)

# Bob computes secret (Using M, thinking it is Alice's A)
kb = power(M, b, P)

# Mallory computes secrets
# Shared with Alice: uses A and m
k_mal_alice = power(x, m, P)
# Shared with Bob: uses B and m
k_mal_bob = power(y, m, P)

print("--- FINAL SECRETS ---")
print(f"Alice's Secret Key (with 'Bob'): {ka}")
print(f"Bob's Secret Key (with 'Alice'): {kb}")
print("-" * 30)
print(f"Mallory's Secret with Alice: {k_mal_alice}")
print(f"Mallory's Secret with Bob:   {k_mal_bob}")

# Verification
assert ka == k_mal_alice
assert kb == k_mal_bob
print("\n[CONCLUSION]: Mallory successfully established separate keys with both parties.")
if ka != kb:
    print("SUCCESS: Alice and Bob have DIFFERENT keys, but Mallory knows both!")
else:
    print("FAILED: Alice and Bob accidentally have the same key.")


## 2. Analysis and Conclusion

As demonstrated in the output:
1.  **Discrepancy**: Alice and Bob computed different secret keys ($k_a \neq k_b$). This means they cannot verify each other's messages if they used these keys for encryption directly, or worse, Mallory translates between them.
2.  **Compromise**: Mallory possesses both keys. $k_{\text{Mal-Alice}}$ matches Alice's secret, and $k_{\text{Mal-Bob}}$ matches Bob's secret.
3.  **Fatal Flaw**: The Standard Diffie-Hellman protocol calculates $K = B^a \pmod p = A^b \pmod p$ based on the assumption that $A$ and $B$ are authentic. Without a digital signature or a trusted certificate authority (CA) to bind the public keys to the identities of Alice and Bob, the protocol cannot distinguish between a legitimate user and an active attacker injecting their own parameters.
